In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
# Cargar datos de entrenamiento
train = pd.read_csv('train.csv')
train[train.columns[1:]] = train[train.columns[1:]].astype(np.float32)
train.fillna(0, inplace=True)

# Remover outliers
train = train[train['Expected'] < 108]

# Agrupar por Id
train_groups = train.groupby("Id")
train_size = len(train_groups)
MAX_SEQ_LEN = train_groups.size().max()
N_FEATURES = 21

# Crear X_train y y_train
X_train = np.zeros((train_size, MAX_SEQ_LEN, N_FEATURES), dtype=np.float32)
y_train = np.zeros(train_size, dtype=np.float32)

i = 0
for _, group in train_groups:
    X = group.values
    seq_len = X.shape[0]
    X_train[i, :seq_len, :] = X[:, 2:23]
    y_train[i] = X[0, 23]
    i += 1


In [3]:
# Dividir datos de entrenamiento en entrenamiento y validación
X_train_rf, X_val_rf, y_train_rf, y_val_rf = train_test_split(X_train.reshape((X_train.shape[0], -1)), y_train, test_size=0.2, random_state=42)

In [4]:
# Crear modelo de Random Forest con parámetros ajustados para velocidad
rf_model = RandomForestRegressor(
    n_estimators=50,       # Reducir el número de árboles
    max_depth=10,          # Limitar la profundidad máxima de los árboles
    max_features='sqrt',   # Utilizar una fracción de las características en cada división
    max_samples=0.8,       # Usar una fracción de los datos para cada árbol
    n_jobs=-1,             # Usar todos los núcleos de la CPU disponibles
    random_state=42
)


In [5]:
# Entrenar el modelo
rf_model.fit(X_train_rf, y_train_rf)

RandomForestRegressor(max_depth=10, max_features='sqrt', max_samples=0.8,
                      n_estimators=50, n_jobs=-1, random_state=42)

In [6]:
# Predecir en datos de validación
y_pred_rf = rf_model.predict(X_val_rf)

In [7]:
# Calcular el error absoluto medio en validación
mae_rf = mean_absolute_error(y_val_rf, y_pred_rf)
print(f'MAE en validación (Random Forest): {mae_rf}')

MAE en validación (Random Forest): 3.971944352351299


In [11]:
# Preparar datos de prueba
test_df = pd.read_csv('test.csv')
test_df[test_df.columns[1:]] = test_df[test_df.columns[1:]].astype(np.float32)
test_df.fillna(0, inplace=True)
test_groups = test_df.groupby("Id")
test_size = len(test_groups)

# Obtener IDs de test
test_ids = list(test_groups.groups.keys())

# Crear X_test
X_test = np.zeros((test_size, MAX_SEQ_LEN, N_FEATURES), dtype=np.float32)

i = 0
for _, group in test_groups:
    X = group.values
    seq_len = X.shape[0]
    X_test[i, :seq_len, :] = X[:, 2:23]
    i += 1

# Predecir en datos de prueba
y_pred_test_rf = rf_model.predict(X_test.reshape((X_test.shape[0], -1)))

In [12]:
# Crear archivo de sumisión
submission = pd.DataFrame({'Id': test_ids, 'Expected': y_pred_test_rf})
submission.to_csv('sample_submission.csv', index=False)